In [150]:
import enum
import typing
from dataclasses import dataclass,field
import numpy as np

import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cmasher as cmr

import networkx as nx
from collections import Counter

In [36]:
def getfallingbricks(filename):
    tower = []
    with open(filename) as f:
        for id,line in enumerate(f):
            brick = np.array([[int(coord) for coord in point.split(",")] for point in line.rstrip().split("~")])
            tower.append(Brick(id,brick))
    return sorted(tower)

In [64]:
def getOverlap(a, b):
    return max(0, min(a[1], b[1])+1 - max(a[0], b[0]))

def settle(tower):
    tower.sort()
    for k,brick in tqdm.tqdm(enumerate(tower)):
        while not brick.checkstable(tower[:k]):
            brick.fall(tower[:k])

@dataclass(eq=True)
class Brick:
    id : int
    coords : np.array
    stable :bool =False
    supportedby : set = field(default_factory=set)

    def __lt__(self,b):
        if self.zmin<b.zmin: return True
        elif self.zmin==b.zmin and self.zmax<b.zmax: return True
        elif (self.z==b.z).all(): return self.id<b.id
        else: return False

    @property
    def x(self):
        return self.coords[:,0]
    @property
    def y(self):
        return self.coords[:,1]
    @property
    def z(self):
        return self.coords[:,2]
    
    @property
    def zmax(self): return self.coords[1,2]

    @property
    def zmin(self): return self.coords[0,2]

    
    def fall(self,tower):
        if not self.stable:
            self.coords[:,2] -=1

    def checkstable(self,tower):
        """Return False if no other brick/ground supports it else the brick or True if on the ground"""
        if self.stable: return True
        else:
            if self.zmin==1: 
                self.stable = True
                return True
        for brick in tower:
            if brick.stable:
                if brick.id!=self.id: # actually not necessary
                    if self.issupportedby(brick):
                        self.supportedby.add(brick.id)
                        self.stable = True
                        return brick.id
        return False

    def overlap(self,brick):
        xover = getOverlap(self.x,brick.x)
        yover = getOverlap(self.y,brick.y)
        return (xover*yover)>0

    def issupportedby(self,brick):
        if self.zmin!=brick.zmax+1: return False
        return self.overlap(brick)



# Part 1

In [65]:
tower = getfallingbricks("input22.txt")
settle(tower) #Super not satisfying

1477it [00:28, 52.56it/s] 


In [140]:
def createsupportbygraph(tower):
    graph = {}
    for b in tower:
        for b2 in tower:
            if b.issupportedby(b2):
                b.supportedby.add(b2.id)
        graph[b.id] = b.supportedby
        if not graph[b.id]:
            graph[b.id] = {-1}
    return  nx.DiGraph(graph)

In [141]:
supportbygraph = createsupportbygraph(tower)
supportgraph = supportbygraph.reverse()

In [147]:
def disintegrablebricks(tower):
    suppby = createsupportbygraph(tower)
    supp = suppby.reverse() 
    disintegrable = []
    for k in supp.nodes():
        if k!=-1:
            if len(supp[k])==0:
                disintegrable.append(k)
            else:
                dualsupport = True
                for j in supp[k]:
                    if len(suppby[j])<2:
                        dualsupport = False
                if dualsupport:
                    disintegrable.append(k)
    return disintegrable
    

In [148]:
disbricks = disintegrablebricks(tower)

In [149]:
len(disbricks)

503

# Part 2

In [ ]:
Counter([len(supportgraph[v]) for v in supportgraph])

Counter({1: 698, 0: 379, 2: 325, 3: 67, 4: 7, 5: 1})

In [ ]:
Counter([len(supportbygraph[v]) for v in supportgraph])

Counter({1: 1335, 2: 113, 0: 22, 3: 7})

In [153]:
def findfallingbricks(support,supportby,brick,fallingbricks):
    fallingbricks.add(brick)
    for overbrick in support[brick]:
        if all([supbrick in fallingbricks for supbrick in supportby[overbrick]]):
            fallingbricks.add(overbrick)
            findfallingbricks(support,supportby,overbrick,fallingbricks)
    return len(fallingbricks)-1
            

In [160]:
sum([findfallingbricks(supportgraph,supportbygraph,b.id,set()) for b in tower])

98431